In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import *
from tkcalendar import DateEntry  # Import the DateEntry widget from tkcalendar
from datetime import datetime
import json

import os
import sys

file_dir = os.path.dirname('/Users/valentinagualt/PycharmProjects/Gualtieri/gualtieri/auxiliaryFunctions.py')
sys.path.append(file_dir)

from auxiliaryFunctions import *



# Sample data for the time slots
time_slots = load_timeslots()
# Sample list of possibilities
activities = load_activities(path='/Users/valentinagualt/PycharmProjects/Gualtieri/activities.txt')

# Load the existing JSON data from the file (if it exists)
file_name = "/Users/valentinagualt/PycharmProjects/Gualtieri/saved_data.json"  # Replace with the name of your JSON file
data_saved = load_json_data(file_name)

# Data storage for saving
data_to_save = {}


def save_data():
    global data_to_save

    # Get the selected date
    selected_date = date_var.get()
    print(selected_date)

    selected_activities = []

    # get a list of the selected activities
    for row, time_slot in enumerate(time_slots, start=0):
        activity = activity_vars[row].get()
        selected_activities.append(activity)

    activity_n_timeslots = count_element_repetitions(selected_activities)
    for case in activity_n_timeslots:
        activity = case[0]
        if activity != 'None':
            n_timeslots = case[1]

            activity_decomposed = activity.split('/')
            project_code = activity_decomposed[0]
            scope_code = 'Undefined'
            task_code = 'Undefined'

            if len(activity_decomposed) == 2:
                scope_code = activity_decomposed[1]

            elif len(activity_decomposed) == 3:
                scope_code = activity_decomposed[1]
                task_code = activity_decomposed[2]

            total_time_on_activity = n_timeslots / 2 #assuming each timeslot half an hour

            data_to_save = {
                f'{project_code}':{
                    f'{scope_code}':{
                        f'{task_code}':{
                            'dates': [selected_date],
                            'duration': [total_time_on_activity]

                        }
                    }
                }
            }

            # Save the data to a JSON file
            with open("/Users/valentinagualt/PycharmProjects/Gualtieri/saved_data.json", "a") as file:
                json.dump(data_to_save, file)

    print("Data saved successfully!")


def create_entry_gui():
    root = tk.Tk()
    root.title("Data Entry GUI")

    # Create a frame for the data entry table
    entry_frame = ttk.Frame(root)
    entry_frame.pack(padx=10, pady=10)

    # Get the current date
    current_date = datetime.now().date()

    # Create the header for the table
    ttk.Label(entry_frame, text="Time Slot").grid(row=0, column=0)

    # Create a DateEntry widget for the right column header to select the date
    global date_var
    date_var = tk.StringVar()
    date_var.set(current_date.strftime("%d/%m/%Y"))
    date_entry = DateEntry(entry_frame, width=12, date_pattern="dd/mm/yyyy", textvariable=date_var)
    date_entry.grid(row=0, column=1)

    # Create a canvas with scrollbar to make the time slots accessible
    canvas = tk.Canvas(entry_frame)
    canvas.grid(row=1, column=0, columnspan=2)
    scrollbar = ttk.Scrollbar(entry_frame, orient="vertical", command=canvas.yview)
    scrollbar.grid(row=1, column=2, sticky="ns")
    canvas.configure(yscrollcommand=scrollbar.set)

    # Create a frame inside the canvas to hold the time slots and activities
    inner_frame = ttk.Frame(canvas)
    canvas.create_window((0, 0), window=inner_frame, anchor="nw")

    # Function to update the scrollable region of the canvas
    def configure_scroll_region(event):
        canvas.configure(scrollregion=canvas.bbox("all"))

    inner_frame.bind("<Configure>", configure_scroll_region)

    # Create the rows for data entry with time slots on the left and dropdown on the right

    global activity_vars
    activity_vars = []

    for row, time_slot in enumerate(time_slots, start=1):
        ttk.Label(inner_frame, text=f"{time_slot}", anchor="center").grid(row=row, column=0)

        # Create a dropdown menu for selecting among the list of activities
        activity_var = tk.StringVar()
        activity_var.set(activities[0])  # Set the default option
        activity_dropdown = ttk.OptionMenu(inner_frame, activity_var, *activities)
        activity_dropdown.grid(row=row, column=1)

        activity_vars.append(activity_var)


    # Add a 'Save' button to save the data
    save_button = ttk.Button(entry_frame, text="Save", command=save_data)
    save_button.grid(row=len(time_slots) + 1, column=1)

    root.mainloop()


if __name__ == "__main__":
    create_entry_gui()

25/07/2023
Data saved successfully!


In [ ]:
import json
from datetime import datetime

# Function to convert date string to datetime object
def string_to_datetime(date_str):
    return datetime.strptime(date_str, "%d/%m/%Y")

# Sample date string
date_string = "25/07/2023"

# Convert date string to datetime object
date_time_element = string_to_datetime(date_string)

# Sample data to save
data_to_save = {"date": date_time_element}

# Save data to a JSON file
with open("saved_data.json", "w") as file:
    json.dump(data_to_save, file)

print("Data saved successfully!")

In [1]:
import json

data = {}  # Initialize an empty dictionary to store the data

# Function to add a new activity, task, or subtask
def add_entry(project_code, activity, task, sub_task, dates, time_slots):
    if project_code not in data:
        data[project_code] = {}
    if activity not in data[project_code]:
        data[project_code][activity] = {}
    if task not in data[project_code][activity]:
        data[project_code][activity][task] = {}
    data[project_code][activity][task][sub_task] = {
        "dates": dates,
        "time_slots": time_slots
    }

# Sample interactive input
project_code = "project_code_1"
activity = "activity_1"
task = "task_1"
sub_task = "sub_task_1"
dates = ["25/07/2023", "26/07/2023"]
time_slots = ["09:00", "10:30"]

# Call the function to add the entry to the data
add_entry(project_code, activity, task, sub_task, dates, time_slots)

# Save the data to a JSON file
with open("project_data.json", "w") as file:
    json.dump(data, file)

# To load the data from the JSON file back into the dictionary
with open("project_data.json", "r") as file:
    data = json.load(file)

['None', 'PHD/QD/Coding', 'PHD/QD/Studying_Papers', 'PHD/Meetings', 'SelfDev/Entr/Reading', 'SelfDev/Lang', 'SelfDev/Sport', 'Socials/Writing_Posts', 'Socials/Others', 'House_Chores']


In [3]:
import os
import sys

file_dir = os.path.dirname('/Users/valentinagualt/PycharmProjects/Gualtieri/gualtieri/auxiliaryFunctions.py')
sys.path.append(file_dir)

from auxiliaryFunctions import *
file_name = "/Users/valentinagualt/PycharmProjects/Gualtieri/saved_data.json"  # Replace with the name of your JSON file

empty_line = {"None": {"Undefined": {"Undefined": {"dates": [], "duration": []}}}}

dic = load_json_data(file_name)
update_data_entry(dic, 'None','Undefined','Undefined', 'blabla', 'balbal',file_name)

None Undefined
Data updated successfully!
